# steam_games

In [8]:
from extract import jsongz

#df_steam_game_original = jsongz.leer_archivo(".\\data_raw\\steam_games.json.gz")
df_original = jsongz.leer_archivo("../data/raw/steam_games.json.gz")

In [60]:
df = df_original.copy()
df.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
df.shape[0]

120445

El df contiene 120445 registros

Evaluar los tipos de datos usados por las columnas

In [63]:
from transform import info

info.mostrar_info_key_types(df)

genres - {<class 'float'>, <class 'list'>}
app_name - {<class 'str'>, <class 'float'>}
title - {<class 'str'>, <class 'float'>}
release_date - {<class 'str'>, <class 'float'>}
id - {<class 'str'>, <class 'float'>}


Se elimina las columnas que no son relevantes para los objetivos del proyecto. Esto está desarrollado en una hoja de cálculo, en una matriz de objetivos vs. datos.

https://docs.google.com/spreadsheets/d/1ccOxyOM_yctvpk4-CbpX4kCihNn1rX1QifKjbtov10s/edit?usp=sharing

In [62]:
from transform import limpiar

limpiar.eliminar_columnas_innecesarias(
    df,
    [
        "publisher",
        "url",
        "tags",
        "reviews_url",
        "specs",
        "price",
        "early_access",
        "developer",
    ],
)
df.head()

,genres,app_name,title,release_date,id
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN


Recordar que las columnas que contiene valores iguales a "float", podria indica que contine valores nulos.

Las columnas que utilizan el tipo de dato "list" deben ser desanidados

In [64]:
from transform import normalizar

df = normalizar.desanidar_dataframe(df, ["genres"])
df.head()

,genres,app_name,title,release_date,id
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN


## Variables
Las que utiliza este daframe son:

### Variables categóricas
* genres - variable categóricas nominales
* app_name - variable categórica nominal
* title - variable categórica nominal
* id - variable categórica nominal (identificador)

### Variables temporal
* release_date - variable temporal

In [66]:
df.shape[0]

163147

Ahora. la cantidad de registros es de 163147

## Manejando la columna "release_date"

Reviso la cantidad de registro de fecha que considero invalidas

In [95]:
from datetime import datetime

count = 0
for i in df_steam_game["release_date"]:
    try:
        datetime.strptime(str(i), "%Y-%m-%d")
    except ValueError:
        print(i)
        count += 1
print(f"Cantidad de registros formato invalido de fecha: {count}")

nan
Soon..
Soon..
Soon..
Soon..
nan
nan
nan
nan
nan
nan
nan
2017
2017
2017
2017
Beta测试已开启
Beta测试已开启
Beta测试已开启
Beta测试已开启
nan
nan
nan
Jun 2009
Jun 2009
Oct 2010
Oct 2010
Oct 2010
Oct 2010
Oct 2010
Oct 2010
Feb 2011
nan
nan
nan
nan
nan
nan
nan
Aug 2014
nan
nan
nan
Aug 2014
Aug 2014
Aug 2014
Aug 2014
Aug 2014
Sep 2014
Sep 2014
Sep 2014
Sep 2014
Sep 2014
Sep 2014
nan
nan
nan
nan
nan
nan
Apr 2015
Apr 2015
Apr 2015
Apr 2015
Apr 2015
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
2017
2017
2017
2017
2017
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
Apr 2016
Apr 2016
Apr 2016
Apr 2016
Apr 2016
Apr 2016
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
Jul 2016
nan
nan
June 2016
June 2016
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
Coming Soon
Coming

Ahora, filtro y convierto el tipo de dato "release_date"

In [96]:
from datetime import datetime

def obtener_fecha(fecha):
    try:
        return datetime.strptime(str(fecha), "%Y-%m-%d")
    except ValueError:
        try:
            return datetime.strptime(str(fecha), "%Y")
        except ValueError:
            return float("nan")

df_steam_game['release_date'] = df_steam_game['release_date'].apply(obtener_fecha)

Verifico el filtro y reemplazo los valores invalidos.

In [97]:
from datetime import datetime

count = 0
for i in df_steam_game["release_date"]:
    try:
        datetime.strptime(str(i), "%Y-%m-%d %H:%M:%S")
    except ValueError:
        print(i)
        count += 1
print(f"Cantidad de registros formato invalido de fecha: {count}")

NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT


Como los datos que hay en "release_date" son fechas completas, lo convertire en otro tipo de dato, pero solo utilizare el anio, no necesto ni el mes , ni el dia para las objetivos del proyecto.

In [99]:
def obtener_fecha(fecha):
    return str(fecha.year)

df_steam_game["release_date"] = df_steam_game["release_date"].apply(obtener_fecha)

In [100]:
from transform import normalizar

df_steam_game = normalizar.cambiar_nombre_a_una_columna(df_steam_game, "release_date", "release_year")
df_steam_game.head()

,genres,app_name,title,release_year,id
88310,Action,Lost Summoner Kitty,Lost Summoner Kitty,2018,761140
88310,Casual,Lost Summoner Kitty,Lost Summoner Kitty,2018,761140
88310,Indie,Lost Summoner Kitty,Lost Summoner Kitty,2018,761140
88310,Simulation,Lost Summoner Kitty,Lost Summoner Kitty,2018,761140
88310,Strategy,Lost Summoner Kitty,Lost Summoner Kitty,2018,761140


## Manejando la columna "id"

In [101]:
from transform import normalizar

df = normalizar.cambiar_nombre_a_una_columna(df, "id", "item_id")
df.head()

,genres,app_name,title,release_year,item_id
88310,Action,Lost Summoner Kitty,Lost Summoner Kitty,2018,761140
88310,Casual,Lost Summoner Kitty,Lost Summoner Kitty,2018,761140
88310,Indie,Lost Summoner Kitty,Lost Summoner Kitty,2018,761140
88310,Simulation,Lost Summoner Kitty,Lost Summoner Kitty,2018,761140
88310,Strategy,Lost Summoner Kitty,Lost Summoner Kitty,2018,761140


## Manejando las columnas "app_name" y "title"

In [103]:
from transform import normalizar

normalizar.unir_columnas_de_texto(df_steam_game, "app_name", "title")
df_steam_game.head()

,genres,release_year,item_id,name
88310,Action,2018,761140,Lost Summoner Kitty
88310,Casual,2018,761140,Lost Summoner Kitty
88310,Indie,2018,761140,Lost Summoner Kitty
88310,Simulation,2018,761140,Lost Summoner Kitty
88310,Strategy,2018,761140,Lost Summoner Kitty


## Limpieza

Revisamos las celdas vacias en el dataframe.

In [68]:
from transform import info

info.celdas_vacias(df)

,Cantidad,Porcentaje
genres,91593,0.561414
app_name,88313,0.541309
title,90361,0.553862
release_date,90412,0.554175
id,88313,0.541309


In [69]:
from transform import limpiar

limpiar.eliminar_filas_vacias(df)

df.head()

la cantidad de filas en el dataframe es: 163147
despues de eliminar las filas vacias es: 74836


,genres,app_name,title,release_date,id
88310,Action,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,761140
88311,Casual,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,761140
88312,Indie,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,761140
88313,Simulation,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,761140
88314,Strategy,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,761140


In [70]:
from transform import limpiar

limpiar.eliminar_filas_duplicadas(df)

df.head()

la cantidad de filas en el dataframe es: 74836
despues de eliminar las filas repetidas es: 74835


,genres,app_name,title,release_date,id
88310,Action,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,761140
88311,Casual,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,761140
88312,Indie,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,761140
88313,Simulation,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,761140
88314,Strategy,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,761140


Como la columna "item_id" es un identificador, deberia tener valores nulos

In [ ]:
from transform import limpiar

limpiar.eliminar_filas_por_celdas_vacias_en(df, ["item_id"])

df.head()

Volvemos a revisar las celdas vacias en el dataframe

In [ ]:
from transform import info

info.celdas_vacias(df)

## Load

In [106]:
#df_steam_game.to_parquet(".\\data\\steam_games.parquet")
df_steam_game.to_parquet("../data/intermediate/steam_games.parquet")